# Healthcare Survey Responses

Tools to help healthcare providers (e.g., doctors, dentists,  chiropractors, therapists) manage their online reputations can be valueable. Thus, one key data initiative involves assessing the performance of these tools in order to help optimize our products and quantify the value they bring to providers. 

For example, one of our tools is an automated feedback survey sent to patients after their appointments. Practices use a “short” version of the survey during their first 6 months and then must switch to one of two “long” versions after that. The goals of these surveys are to generate 1) more online reviews for a practice and 2) higher ratings. 

The goal of this project is to analyze the effectiveness of each of the “long” versions and recommend which “long” version would be best suited to each practice and specifically, to determine the answer to the following questions:
    - Do the “long” surveys result in significantly more reviews than the “short” surveys? Which “long” survey has a greater impact (v1 or v2)?
    - What kinds of practices do best with “long” survey v1? Would you recommend “long” survey v2 to any specific kinds of practices?
    - Do ratings change significantly with either “long” survey? How would you predict which practices will have success, defined as at least a 0.3 star increase in their “long” survey average monthly rating vs their “short” survey average monthly rating? 
    - Would you recommend replacing the “short” survey with either (or both) of the “long” surveys for any or all practices? Provide reasons to support your answer. If you do not have enough data to make a recommendation, what additional data would you need?

### Introduction

The following Jupyter Notebook analyzes a small subset of response data. The Jupyter Notebook format was chosen because it is a commonly used platform that combines graphs, text, and code. Thereby, allowing others to assess ability to analyze data using analytics, statistics, machine learning methods and to tell informative data stories with visualizations and the insights.

### Importing Packages and Data Sets

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from scipy import stats
import seaborn as sns
from datetime import timedelta, datetime
warnings.filterwarnings('ignore')

In [2]:
# Split Excel into two preferred CSV files
reviews = pd.read_csv("reviews.csv")

### Previewing Data

In [3]:
# List of all Products
reviews.head()

,review_id,review_date,rating,practice_id,n_providers,n_providers_surveys,short_start_date,long_start_date,version,city,state,zip,tech_type,specialty
0,274252,2017-05-23,5.0,2,1,1,2017-04-13,2017-10-10,v1,Santa Monica,CA,90404,ATHEN,Obstetrician and Gynecologist
1,267767,2017-12-30,5.0,2,1,1,2017-04-13,2017-10-10,v1,Santa Monica,CA,90404,ATHEN,Obstetrician and Gynecologist
2,626201,2018-04-03,1.0,2,1,1,2017-04-13,2017-10-10,v1,Santa Monica,CA,90404,ATHEN,Obstetrician and Gynecologist
3,161319,2017-09-08,5.0,3,1,1,2017-09-03,2018-03-02,v2,Santa Monica,CA,90404,ADVAN,Obstetrician and Gynecologist
4,275147,2017-09-11,5.0,3,1,1,2017-09-03,2018-03-02,v2,Santa Monica,CA,90404,ADVAN,Obstetrician and Gynecologist


In [4]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410413 entries, 0 to 410412
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   review_id            410413 non-null  int64  
 1   review_date          410413 non-null  object 
 2   rating               410413 non-null  float64
 3   practice_id          410413 non-null  int64  
 4   n_providers          410413 non-null  int64  
 5   n_providers_surveys  410413 non-null  int64  
 6   short_start_date     410413 non-null  object 
 7   long_start_date      410413 non-null  object 
 8   version              410413 non-null  object 
 9   city                 410413 non-null  object 
 10  state                410413 non-null  object 
 11  zip                  410413 non-null  int64  
 12  tech_type            407466 non-null  object 
 13  specialty            378959 non-null  object 
dtypes: float64(1), int64(5), object(8)
memory usage: 43.8+ MB


Many columns are defined as objects, but some should be converted to dates and integers.

In [5]:
reviews.describe()

,review_id,rating,practice_id,n_providers,n_providers_surveys,zip
count,4.104130e+05,410413.000000,410413.000000,410413.000000,410413.000000,410413.000000
mean,1.143601e+06,3.567973,12097.962935,5.336044,4.238460,53547.366511
std,5.004602e+05,2.172081,1955.647520,6.909282,6.295056,31017.726713
min,9.847100e+04,0.000000,2.000000,1.000000,0.000000,1028.000000
25%,7.317630e+05,1.000000,10796.000000,1.000000,1.000000,23462.000000
50%,1.218286e+06,5.000000,12100.000000,2.000000,1.000000,60181.000000
75%,1.620049e+06,5.000000,13418.000000,6.000000,5.000000,78758.000000
max,1.847052e+06,5.000000,15923.000000,35.000000,33.000000,99701.000000


### Data Cleaning and Manipulation

In [6]:
reviews['version'].unique()

array(['v1', 'v2'], dtype=object)

In [7]:
reviews['version'] = reviews['version'].replace("v1", "1")
reviews['version'] = reviews['version'].replace("v2", "2")
reviews['version'] = reviews['version'].astype('int64')

In [8]:
# Correct 3017 year to 2017
reviews['review_date'].loc[reviews['review_date'] == '3017-05-03'] = '2017-05-03'


In [9]:
reviews['short_start_date'] = reviews['short_start_date'].astype('datetime64[ns]')
reviews['long_start_date'] = reviews['long_start_date'].astype('datetime64[ns]')
reviews['review_date'] = reviews['review_date'].astype('datetime64[ns]')

Since version contains a long version even if the survey was a short version at the time; version_type will display actual type of survey that is being recorded.

In [10]:
# 0 = short, 1 = long v1, 2 = long v2
reviews['review_type'] = np.nan
for i in range(len(reviews)):
    if reviews['review_date'][i] < reviews['long_start_date'][i]:
        reviews['review_type'][i] = 0
    elif reviews['review_date'][i] >= reviews['long_start_date'][i] and reviews['version'][i] == 1:  
        reviews['review_type'][i] = 1
    elif reviews['review_date'][i] >= reviews['long_start_date'][i] and reviews['version'][i] == 2:
        reviews['review_type'][i] = 2

In [11]:
backup = reviews

In [12]:
reviews[reviews['rating'] == 0]

,review_id,review_date,rating,practice_id,n_providers,n_providers_surveys,short_start_date,long_start_date,version,city,state,zip,tech_type,specialty,review_type
262,1589790,2018-05-14,0.0,3,1,1,2017-09-03,2018-03-02,2,Santa Monica,CA,90404,ADVAN,Obstetrician and Gynecologist,2.0
263,1589789,2018-05-14,0.0,3,1,1,2017-09-03,2018-03-02,2,Santa Monica,CA,90404,ADVAN,Obstetrician and Gynecologist,2.0
264,1719975,2018-05-15,0.0,3,1,1,2017-09-03,2018-03-02,2,Santa Monica,CA,90404,ADVAN,Obstetrician and Gynecologist,2.0
265,1719976,2018-05-15,0.0,3,1,1,2017-09-03,2018-03-02,2,Santa Monica,CA,90404,ADVAN,Obstetrician and Gynecologist,2.0
266,1724256,2018-05-16,0.0,3,1,1,2017-09-03,2018-03-02,2,Santa Monica,CA,90404,ADVAN,Obstetrician and Gynecologist,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409859,1701098,2018-12-12,0.0,15780,7,7,2018-06-29,2018-12-26,2,Chesterfield,MI,48047,GREEN,Podiatrist,0.0
410016,1701058,2018-11-14,0.0,15783,2,1,2018-06-14,2018-12-11,2,Norwell,MA,2061,SIKKA,General Dentist,0.0
410017,1721564,2018-12-14,0.0,15783,2,1,2018-06-14,2018-12-11,2,Norwell,MA,2061,SIKKA,General Dentist,2.0
410020,1725027,2018-12-16,0.0,15783,2,1,2018-06-14,2018-12-11,2,Norwell,MA,2061,SIKKA,General Dentist,2.0


There are 99188 entries without ratings.

In [13]:
non_zero = reviews[reviews['rating'] != 0]
non_zero[['rating','practice_id']].groupby('practice_id').count().describe()

,rating
count,3807.000000
mean,81.750722
std,256.214132
min,1.000000
25%,14.000000
50%,34.000000
75%,73.000000
max,8175.000000


The average practice has 81.75 reviews but the median count is 34 reviews. The 25% quartile value of 14 will be used as the cutoff to apply the practice average to the 0 values. For those less than 14, 0 rating values will be set to NaN.

In [14]:
greater_equal_14 = practice_rating_cnt[practice_rating_cnt['rating_count']  >= 14]['practice_id_'].unique()


NameError: name 'practice_rating_cnt' is not defined

In [ ]:
# Set 0 Rating to Practice Average or NaN
for i in range(len(reviews)):
    if reviews['rating'][i] == 0 and reviews['practice_id'][i] in greater_equal_14:
        reviews['rating'][i] = practice_rating_cnt[practice_rating_cnt['practice_id_'] == reviews['practice_id'][i]]['rating_mean']
    elif reviews['rating'][i] == 0:
        reviews['rating'][i] = np.nan
    
    

In [ ]:
# Perform an assessment of how much missing data there is in each column of the
# dataset.

# Shape returns the max number of rows; count returns number of rows with values
reviews.shape[0] - reviews.count()

For the questions, missing tech type data is unimportant. Additionally, missing speciality data can still be used for the non-speciality questions.

In [ ]:
# How much data is missing in each row of the dataset?
arr = reviews.shape[1]-reviews.count(axis=1)
arr.describe()

In [ ]:
reviews = reviews[pd.notnull(reviews['rating'])]

In [ ]:
# There number of entries does not match the number of unique IDs; this will be treated as acceptable
reviews['review_id'].nunique()

In [ ]:
practice_rating_cnt = reviews[reviews['rating'] != 0][['rating','practice_id']].groupby('practice_id').describe()
practice_rating_cnt = practice_rating_cnt.reset_index()
practice_rating_cnt.columns = ['_'.join(col).strip() for col in practice_rating_cnt.columns.values]
practice_rating_cnt.head()


In [ ]:
reviews.info()

In [ ]:
reviews.describe()

### Question 1
#### Do the “long” surveys result in significantly more reviews than the “short” surveys? Which “long” survey has a greater impact (v1 or v2)?

You can not technically say if long surveys result in more reviews than short surveys because this is a sample dataset and it is unknown if the sample dataset was constructed randomly or constructed weightedly/biasedly. Yet, if we assume this is a population dataset or an equally representative, random, statistically significant dataset, then the long format had the most reviews (329,755; 80.35%) and within the long format v2 had the most reviews (45.08%) (including reviews with 0 ratings). If you only look at the data after some of the 0 ratings have been dropped, then the percentages are: XXXXXXX

It is important to note that these initial numbers are not causation - they do not prove that having a long format does not cause a user to review more.

In [ ]:
backup.groupby('review_type')['rating'].count()

In [ ]:
reviews.groupby('review_type')['rating'].count()

In [ ]:
reviews.groupby(['practice_id','review_type'])['rating'].describe().head(10)

Glancing through the above data it looks like there are more survey responses for 1 and 2 than there was for 0 (short); however, this still does not prove a causation relationship. You should also restrict the time durations to be equal to ensure the long survey numbers are not inflated. Thus, it would be 6 months of both short and long.

### Question 2
#### What kinds of practices do best with “long” survey v1? Would you recommend “long” survey v2 to any specific kinds of practices?

Since there are large standard deviations relative to the mean for the ratings, the standard deviation will not be used when determining the recommendation of v1 or v2.

3 specialities had the highest mean while on the short survey. 11 did best on Long v1 and 46 did best on Long v2. The specific specialities are listed below.

In [ ]:
q2 = reviews.groupby(['specialty','review_type'])['rating'].describe()
q2 = q2.reset_index()
q2.columns = [''.join(col).strip() for col in q2.columns.values]


In [ ]:
q2.head()

In [ ]:
v1_btr = []
v2_btr = []
s_btr = []
for i in range(len(q2)):
    if q2[(q2['specialty'] == q2['specialty'][i]) & (q2['review_type'] != q2['review_type'][i])]['review_type'].nunique() == 1:
        if float(q2['mean'][i]) > float(q2[(q2['specialty'] == q2['specialty'][i]) & (q2['review_type'] != q2['review_type'][i])]['mean']):
                if q2['review_type'][i] == 0:
                    s_btr.append(q2['specialty'][i])
                elif q2['review_type'][i] == 1.0:
                    v1_btr.append(q2['specialty'][i])
                elif q2['review_type'][i] == 2.0:
                    v2_btr.append(q2['specialty'][i])
    elif q2['review_type'][i] != 0:
        if float(q2['mean'][i]) > float(q2[(q2['specialty'] == q2['specialty'][i]) & (q2['review_type'] != q2['review_type'][i]) & (q2['review_type'] != 0)]['mean']):
            if q2['review_type'][i] == 1.0:
                v1_btr.append(q2['specialty'][i])
            elif q2['review_type'][i] == 2.0:
                v2_btr.append(q2['specialty'][i])
                
        

In [ ]:
s_best_for_specialty = pd.DataFrame(np.column_stack([s_btr]), columns=['Short'])
s_best_for_specialty

In [ ]:
v1_best_for_specialty = pd.DataFrame(np.column_stack([v1_btr]), columns=['Long v1'])
v1_best_for_specialty

In [ ]:
v2_best_for_specialty = pd.DataFrame(np.column_stack([v2_btr]), columns=['Long v2'])
v2_best_for_specialty
        

### Question 3
#### Do ratings change significantly with either “long” survey? How would you predict which practices will have success, defined as at least a 0.3 star increase in their “long” survey average monthly rating vs their “short” survey average monthly rating?

No rating changes statistically significantly for either long survey - the standard deviations are large respective to the values of the ratings. 

In [ ]:
q3 = reviews.groupby(['practice_id','review_type'])['rating'].describe()
q3 = q3.reset_index()
q3.columns = [''.join(col).strip() for col in q3.columns.values]


In [ ]:
q3['practice_id'].count(2)

In [ ]:
for i in range(len(q3))
    if q3['practice_id']

In [ ]:
i

In [ ]:

float(q3[(q3['practice_id'] == q3['practice_id'][i]) & (q3['review_type'] == 0)]['mean'])

In [ ]:
v1_3 = []
v2_3 = []

for i in range(len(q3)):
    if q3['review_type'][i] != 0:
        if float(q3['mean'][i]) > (0.3 + float(q3[(q3['practice_id'] == q3['practice_id'][i]) & (q3['review_type'] == 0)]['mean'])):
            if q3['review_type'][i] == 1.0:
                v1_btr.append(i)
            elif q3['review_type'][i] == 2.0:
                v2_btr.append(i)

In [ ]:
bar = reviews[]
bar = reviews.groupby(['specialty','review_type']).describe()

# Flattening Hierarchical Index
bar = bar.reset_index()
bar.columns = ['_'.join(col).strip() for col in stacked_bar.columns.values]
bar = stacked_bar.drop(stacked_bar.columns[[0]], axis=1)


bar

There are differences between people who switched and those who did not switch having a higher or lower mean retention period depending on product. In other words, for example, switchers to product X had lower retention periods than non-switchers and switchers to product Y had higher retention periods than non-switches.

In [ ]:
# group by product first then switched or not
stacked_bar = small[['viewable_product_id','to_viewable_product_id','switched','max_date_diff']]
stacked_bar = stacked_bar.dropna(subset=['max_date_diff'])
stacked_bar['to_viewable_product_id'] = stacked_bar['to_viewable_product_id'].fillna(0)
stacked_bar = stacked_bar.groupby(['viewable_product_id','switched','to_viewable_product_id']).describe()
stacked_bar = stacked_bar.reset_index()

# Irrelevant Subscription Ends
sub_end = []
for i in stacked_bar[(stacked_bar['to_viewable_product_id'] == 0) & (stacked_bar['switched'] == 1)].index:
    sub_end.append(i)
stacked_bar = stacked_bar.drop(stacked_bar.index[sub_end])

# Flattening Hierarchical Index
stacked_bar = stacked_bar.reset_index()
stacked_bar.columns = ['_'.join(col).strip() for col in stacked_bar.columns.values]
stacked_bar = stacked_bar.drop(stacked_bar.columns[[0]], axis=1)

# Exporting Data
# stacked_bar.to_csv('stacked_bar.csv')


In [ ]:
pairings_higher_retention = []
pairings_lower_retention = []
for i in range(len(stacked_bar)):
    if stacked_bar['switched_'][i] == 1:
        # Switchers Statitistically Significant Higher
        if stacked_bar['max_date_diff_count'][i] > 1:
            h_pairing = ((stacked_bar['max_date_diff_mean'][i] - stacked_bar['max_date_diff_std'][i]) / np.timedelta64(1, 'D'))
        else:
            h_pairing = ((stacked_bar['max_date_diff_mean'][i]) / np.timedelta64(1, 'D'))
        # Switchers Statitistically Significant Lower
        if stacked_bar['max_date_diff_count'][i] > 1:
            l_pairing = ((stacked_bar['max_date_diff_mean'][i] + stacked_bar['max_date_diff_std'][i]) / np.timedelta64(1, 'D'))
        else:
            l_pairing = ((stacked_bar['max_date_diff_mean'][i]) / np.timedelta64(1, 'D'))
        # Switchers Statitistically Significant Higher   
        if stacked_bar[(stacked_bar['viewable_product_id_'] == stacked_bar['viewable_product_id_'][i]) & 
                 (stacked_bar['switched_'] == 0)]['max_date_diff_count'].values > 1:
            h_non_switchers = ((stacked_bar[(stacked_bar['viewable_product_id_'] == stacked_bar['viewable_product_id_'][i]) & 
                 (stacked_bar['switched_'] == 0)]['max_date_diff_mean'] + stacked_bar[(stacked_bar['viewable_product_id_'] == stacked_bar['viewable_product_id_'][i]) & 
                  (stacked_bar['switched_'] == 0)]['max_date_diff_std']) / np.timedelta64(1, 'D')).values
        else: 
            h_non_switchers = ((stacked_bar[(stacked_bar['viewable_product_id_'] == stacked_bar['viewable_product_id_'][i]) & 
                 (stacked_bar['switched_'] == 0)]['max_date_diff_mean']) / np.timedelta64(1, 'D')).values
        # Switchers Statitistically Significant Lower
        if stacked_bar[(stacked_bar['viewable_product_id_'] == stacked_bar['viewable_product_id_'][i]) & 
                 (stacked_bar['switched_'] == 0)]['max_date_diff_count'].values > 1:
            l_non_switchers = ((stacked_bar[(stacked_bar['viewable_product_id_'] == stacked_bar['viewable_product_id_'][i]) & 
                 (stacked_bar['switched_'] == 0)]['max_date_diff_mean'] - stacked_bar[(stacked_bar['viewable_product_id_'] == stacked_bar['viewable_product_id_'][i]) & 
                  (stacked_bar['switched_'] == 0)]['max_date_diff_std']) / np.timedelta64(1, 'D')).values
        else: 
            l_non_switchers = ((stacked_bar[(stacked_bar['viewable_product_id_'] == stacked_bar['viewable_product_id_'][i]) & 
                 (stacked_bar['switched_'] == 0)]['max_date_diff_mean']) / np.timedelta64(1, 'D')).values
        # Comparisons
        if h_pairing > h_non_switchers:
            pairings_higher_retention.append(i)
        if l_pairing < l_non_switchers:
            pairings_lower_retention.append(i)
            

In [ ]:
stacked_bar.loc[pairings_higher_retention].reset_index()

In [ ]:
stacked_bar.loc[pairings_lower_retention].reset_index()

**Summary**

When viewed as solely comparing switched to non-switched, there was not a statistically significant difference between the max retention period of the two groups. However, when dividing the subscriptions into groups of what was being switched from and to, 26 from-to subscription pairings of people who switched was statistically higher than those who did not switch. Also 10 from-to subscription pairings of people who switched was statistically lower than those who did not switch. Yet, this was on limited data - many pairings only had 1 data point. Thus, these numbers might not hold up with a more sizable dataset.

### Question 4
#### Would you recommend replacing the “short” survey with either (or both) of the “long” surveys for any or all practices? Provide reasons to support your answer. If you do not have enough data to make a recommendation, what additional data would you need?

# Additional Analysis

This section one-hot encodes categorical variables so that these variables can be included when calculating the Spearman correlation and R value. The encodings can also be put into a future machine learning model to predict retention. One-hot encoding is breaking a catergorical variable into seperate binary variables for each category. For example, a 'housepet' column would have the options cat and dog. Instead of one column with both cat and dog, you get two columns. One labeled cat and one labeled dog with 'yes' or 'no' for the options.

In [ ]:
# One Hot Encoding to Convert Categorical to Numerical Indicator Variables
correl = small
drop_ar = ['starter_set_count', 'other_set_count', 'handle_count', 'lipbalm_count',
        'razorstand_count','travel_kit_count','abbrev','id',
           'min_start','max_end','created_at','removed_at']
correl = correl.drop(drop_ar, axis=1)
created = pd.get_dummies(correl['created_by_client_type'])
correl = correl.drop('created_by_client_type',axis = 1)
correl = correl.join(created)
correl.rename(columns={'admin':'c_admin'}, inplace=True)
correl.rename(columns={'ios':'c_ios'}, inplace=True)
correl.rename(columns={'system':'c_system'}, inplace=True)
correl.rename(columns={'www':'c_www'}, inplace=True)

removed = pd.get_dummies(correl['removed_by_client_type'])
correl = correl.drop('removed_by_client_type',axis = 1)
correl = correl.join(removed)
correl.rename(columns={'admin':'r_admin'}, inplace=True)
correl.rename(columns={'ios':'r_ios'}, inplace=True)
correl.rename(columns={'system':'r_system'}, inplace=True)
correl.rename(columns={'www':'r_www'}, inplace=True)

In [ ]:
# Day Retention
correl['subs_date_diff'] = (correl['subs_date_diff'])  / np.timedelta64(1,'D')
correl['max_date_diff'] = (correl['max_date_diff'])  / np.timedelta64(1,'D')

In [ ]:
# Spearman Test and P-Value Table
rho, pval = stats.spearmanr(correl, nan_policy='omit')

pvals = pd.DataFrame(pval, columns=['viewable_product_id','quantity','user_id','price','blade_count','shave_gel_count','shave_cream_count','face_wash_count','aftershave_count','face_lotion_count','subs_date_diff','switched','to_viewable_product_id','max_date_diff','c_admin','c_ios','c_system','c_www','r_admin','r_ios','r_www'])
pvals = pvals.rename(index={0: 'viewable_product_id',1: 'quantity',2:'user_id',3:'price',4:'blade_count',5:'shave_gel_count',6:'shave_cream_count',7:'face_wash_count',8:'aftershave_count',9:'face_lotion_count',10:'subs_date_diff',11:'switched',12:'to_viewable_product_id',13:'max_date_diff',14:'c_admin',15:'c_ios',16:'c_system',17:'c_www',18:'r_admin',19:'r_ios',20:'r_www'})
round(pvals, 2)


To be statistically significantly correlated, the p-value must be less than or equal to 0.05. In the below graph, bright yellow denotes items that meet the p-value threshold.

In [ ]:
# Graphing P-Values
mask = np.zeros_like(pvals)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    p2 = sns.heatmap(pvals, mask=mask, square=True, cmap = "YlGn")

In [ ]:
# R Table
rhos = pd.DataFrame(rho, columns=['viewable_product_id','quantity','user_id','price','blade_count','shave_gel_count','shave_cream_count','face_wash_count','aftershave_count','face_lotion_count','subs_date_diff','switched','to_viewable_product_id','max_date_diff','c_admin','c_ios','c_system','c_www','r_admin','r_ios','r_www'])
rhos = rhos.rename(index={0: 'viewable_product_id',1: 'quantity',2:'user_id',3:'price',4:'blade_count',5:'shave_gel_count',6:'shave_cream_count',7:'face_wash_count',8:'aftershave_count',9:'face_lotion_count',10:'subs_date_diff',11:'switched',12:'to_viewable_product_id',13:'max_date_diff',14:'c_admin',15:'c_ios',16:'c_system',17:'c_www',18:'r_admin',19:'r_ios',20:'r_www'})
round(rhos, 3)

In [ ]:
# R Plot
sns.heatmap(rhos, square=True)
plt.yticks(rotation=0)
plt.xticks(rotation=90)

In [ ]:
# Finds Statistically Significant and Strong Correlations
items = ['viewable_product_id','quantity','price','blade_count','shave_gel_count','shave_cream_count','face_wash_count','aftershave_count','face_lotion_count','subs_date_diff','switched','to_viewable_product_id','max_date_diff','c_admin','c_ios','c_system','c_www','r_admin','r_ios','r_www']
arr = []

for i in items:
    for j in items:
        if pvals.loc[i,j] <= 0.05 and (rhos.loc[i,j] > 0.5 or rhos.loc[i,j] < -0.5) and i != j and [j,i, pvals.loc[j,i], rhos.loc[j,i]] not in arr:
            arr.append([i,j, pvals.loc[i,j], rhos.loc[i,j]])

arr = pd.DataFrame(arr,columns=['1','2','P-Value','R'])
arr.sort_values('R', ascending = True)

#### Summary



Having a statistically significant p-value is only half the picture. After having a P-value of less than or equal to 0.05, you also want to have a high R-value which tells that there is a strong correlation. The tests show that
c_admin-c_www, price-shave_gel_count, viewable_product_id-blade_count, price-blade_count, and subs_date_diff-max_date_diff all are statistically significant and strongly correlated. Unfortunately this info is not revolutionary, as each of these variables are based on the other. The price is dependent on shave_gel_count and blade_count, blade_count is dependent on viewable_product_id, and max_date_diff is dependent on subs_date_diff.

# Next Steps

I would re-run the above tests with more data so you can reach a significant, sizable dataset for each product. Once those have been re-run, if the data is still similar, I would then target the switchers who switched to a product that was tied to lower average retention periods. The goal would be to incentivize those at-risk users to subscribe to a product that has an average longer retention period.  

- Verify if no stars given is permissible in their system and is interpretted negatively or is viewed a null
    - In other words, is 1 star better than 0 stars? If 0 stars is better then that means 0 should be ignored and treated as null values
    - In this solution, 0 was treated as null and therefore, upon meeting certain conditions, was replaced with the practice average

- Verify if review_id is unique for a provider or unique throughout the entire review system
    - In this solution, review id was treated as unique to the provider; which means that you could have duplicate review ids

## Additional Ideas for Investigation

1) Investigate how the models change implementing a 0 rating value fill in process that fills in with mean of practice then if not enough surveys in practice, it goes to the mean in speciality, and if not enough in the speciality then the row gets dropped from model inclusion

2) Build a regression model for retention days

3) What other datapoints can we collect about users that could provide insights into their shopping behavior?
    -Type of Credit Card they use (American Express, Discover, Visa, MasterCard...)
    -Computer or Phone they are purchasing from
    -City, State, and Country purchases are originating from

**Note:** No non-ended subscriptions were used because the data is inconclusive. For example, the subscription could be 100 days or 1 day past the data period provided. However, in building a machine learning model, it is common to insert the mean, median, or mode in the null fields.